# Video Generation

> Functions for generating test videos and audio files

In [ ]:
#| default_exp video

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from time import time
import subprocess
import wave
import numpy as np

## Video Generation with Binaural Audio

In [ ]:
#| export
def generate_video_with_binaural_audio(
    filename: str, # Output video filename
    duration: int = 60, # Video duration in seconds
    video_pattern: str = "mandelbrot", # FFmpeg video pattern (mandelbrot, life, etc.)
    wave_type: str = "alpha", # Brainwave type (delta, theta, alpha, beta, gamma)
    base_freq: int = 200, # Base frequency in Hz
    resolution: str = "1920x1080" # Video resolution
) -> bool: # True if video was generated successfully, False otherwise
    """Generate test video with binaural beats audio."""
    
    # Wave type frequencies
    wave_freqs = {
        'delta': 2, 'theta': 6, 'alpha': 10, 'beta': 20, 'gamma': 40
    }
    
    beat_freq = wave_freqs.get(wave_type, 10)
    left_freq = base_freq
    right_freq = base_freq + beat_freq
    
    # Different filter configurations based on pattern
    if video_pattern == "mandelbrot":
        video_filter = f'mandelbrot=size={resolution}:rate=30'
    elif video_pattern == "life":
        video_filter = f'life=size={resolution}:rate=30'
    else:
        # For patterns that support duration parameter
        video_filter = f'{video_pattern}=duration={duration}:size=1920x1080'
    
    cmd = [
        'ffmpeg',
        # Video input - use -t to limit duration
        '-f', 'lavfi', '-i', video_filter,
        '-t', str(duration),  # This controls the duration
        # Left audio channel
        '-f', 'lavfi', '-i', f'sine=frequency={left_freq}:duration={duration}',
        # Right audio channel  
        '-f', 'lavfi', '-i', f'sine=frequency={right_freq}:duration={duration}',
        # Mix audio channels
        '-filter_complex', '[1:a][2:a]join=inputs=2:channel_layout=stereo[a]',
        '-map', '0:v', '-map', '[a]',
        # Encoding
        '-c:v', 'libx264', '-c:a', 'aac', '-b:a', '192k',
        '-pix_fmt', 'yuv420p',
        '-shortest',  # Stop when shortest input ends
        '-async', '1',  # Audio sync
        # Metadata
        '-metadata', f'title={wave_type.title()} Wave Binaural Video',
        '-metadata', f'comment=Binaural beats: {left_freq}Hz left, {right_freq}Hz right',
        '-y', filename
    ]
    
    try:
        start_time = time()
        subprocess.run(cmd, check=True, capture_output=True)
        print(f"Video with binaural audio generated: {filename}")
        print(f"- Wave type: {wave_type} ({beat_freq}Hz)")
        print(f"- Left channel: {left_freq}Hz, Right channel: {right_freq}Hz")
        end_time = time()
        print(f"Process took: {(end_time-start_time):.2f} seconds")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        return False

## Test Audio File Generation

In [ ]:
#| export
def generate_test_audio_file(
    save_path: Path # Path where the test audio file will be saved
) -> None: # No return value
    """Generate a test audio file with binaural beats."""
    # Generate 40Hz binaural beats for 3 minutes
    sample_rate = 44100
    duration = 180  # 3 minutes
    base_frequency = 200  # Base frequency in Hz
    beat_frequency = 40   # Binaural beat frequency in Hz
    
    # Calculate left and right ear frequencies
    left_freq = base_frequency
    right_freq = base_frequency + beat_frequency
    
    # Generate time array
    t = np.linspace(0, duration, int(sample_rate * duration))
    
    # Generate left and right channel sine waves
    left_channel = np.sin(2 * np.pi * left_freq * t)
    right_channel = np.sin(2 * np.pi * right_freq * t)
    
    # Convert to 16-bit integers
    left_channel = (left_channel * 32767 * 0.5).astype(np.int16)  # Reduced volume
    right_channel = (right_channel * 32767 * 0.5).astype(np.int16)
    
    # Interleave left and right channels for stereo
    stereo_audio = np.empty((left_channel.size + right_channel.size,), dtype=np.int16)
    stereo_audio[0::2] = left_channel
    stereo_audio[1::2] = right_channel
    
    # Save as stereo WAV file
    with wave.open(str(save_path), 'w') as wav_file:
        wav_file.setnchannels(2)  # Stereo
        wav_file.setsampwidth(2)  # 2 bytes per sample
        wav_file.setframerate(sample_rate)
        wav_file.writeframes(stereo_audio.tobytes())

    print(f"Generated 40Hz binaural beats:")
    print(f"- Left ear: {left_freq}Hz")
    print(f"- Right ear: {right_freq}Hz")
    print(f"- Beat frequency: {beat_frequency}Hz")
    print(f"- Duration: {duration/60:.1f} minutes")
    print(f"- File saved to: {save_path}")

## Testing

In [ ]:
#|eval: false
from cjm_ffmpeg_utils.core import FFMPEG_AVAILABLE

if FFMPEG_AVAILABLE:
    # Generate test video
    test_video_path = Path("alpha_meditation_video.mp4")
    generate_video_with_binaural_audio(
        test_video_path, 
        duration=5, 
        video_pattern="life", 
        wave_type="alpha", 
        resolution="1280x720"
    )

Video with binaural audio generated: alpha_meditation_video.mp4
- Wave type: alpha (10Hz)
- Left channel: 200Hz, Right channel: 210Hz
Process took: 0.96 seconds


In [ ]:
#|eval: false
# Generate test audio file
test_audio_path = Path("40hz_binaural_beats.wav")
generate_test_audio_file(test_audio_path)

Generated 40Hz binaural beats:
- Left ear: 200Hz
- Right ear: 240Hz
- Beat frequency: 40Hz
- Duration: 3.0 minutes
- File saved to: 40hz_binaural_beats.wav


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()